In [0]:
pip install --upgrade kaggle

In [6]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 50.6MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.04MB/s]


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns

train = pd.read_csv('/content/train.csv', header=0, sep=',', quotechar='"', index_col='PassengerId')
test = pd.read_csv('/content/test.csv', header=0, sep=',', quotechar='"', index_col='PassengerId')
gndr = pd.read_csv('/content/gender_submission.csv', header=0, sep=',', quotechar='"')

In [0]:
train['Age'] = train['Age'].fillna(train['Age'].mode().values[0])   
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode().values[0])
train['Cabin'] = train['Cabin'].fillna("Common")

In [0]:
test['Age'] = test['Age'].fillna(test['Age'].mode().values[0])   
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].mode().values[0])   
test['Cabin'] = test['Cabin'].fillna("Common")   
test['Fare'] = test['Fare'].fillna(test['Fare'].mode().values[0])   

In [0]:
cate = ['Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Embarked','Cabin']

for i in cate:
    if i == 'id':
        continue
    train[i] = train[i].astype('category')
    train[i] = train[i].cat.codes

for i in cate:
    if i == 'id' or i == 'Survived':
        continue
    test[i] = test[i].astype('category')
    test[i] = test[i].cat.codes

In [0]:
# label encoding the data 
from sklearn.preprocessing import LabelEncoder 

le = LabelEncoder()

for a in train.columns:
    train[a]= le.fit_transform(train[a]) 
    test[a] = le.fit_transform(test[a])

In [0]:
train.shape

(891, 11)

In [0]:
test.shape

(418, 10)

In [0]:
train.hist()

In [0]:
CorrMat = train.corr()
plt.gcf().set_size_inches(10,8)
sns.heatmap(CorrMat,annot =True)

In [0]:
dropcol = ['Pclass','Name']

train = train.drop(dropcol,axis = 1)
test = test.drop(dropcol,axis = 1)

In [0]:
y = train['Survived']
x = train.drop(['Survived'],axis = 1)

In [0]:
from sklearn.preprocessing import StandardScaler

x = pd.DataFrame(StandardScaler().fit_transform(x),columns = x.columns)
test = pd.DataFrame(StandardScaler().fit_transform(test),columns = test.columns)

##Modelling

In [0]:
#Functions for moelling and evaluation for Classification

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

modelsClassification = []
modelsClassification.append(('LR ',LogisticRegression()))
# modelsClassification.append(('DTC',DecisionTreeClassifier()))
# modelsClassification.append(('RFC',RandomForestClassifier(n_estimators=200)))
# modelsClassification.append(('KNN',KNeighborsClassifier()))
# modelsClassification.append(('GNB',GaussianNB()))
# modelsClassification.append(('SGD',SGDClassifier()))
# modelsClassification.append(('SVC',SVC()))

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

def ModellingAndEvaluationClassification(xTrain,xTest,yTrain,yTest):
    ConfusionMatries = []
    AccuracyScores = []
    ClassificationReports = []
    for name,model in modelsClassification:
        model.fit(xTrain,yTrain)
        predict = model.predict(xTest)

        if yTest is not None:
            ConfusionMatries.append((name,confusion_matrix(yTest,predict)))
            AccuracyScores.append((name,accuracy_score(yTest,predict)))        
            ClassificationReports.append((name,classification_report(yTest,predict)))
        
    # print('ConfusionMatrices:-')
    # for name, score in ConfusionMatries:
    #     print(name,':')
    #     print(score)
    #     print()
        
    print('Accuracy Scores:-')        
    for name, score in AccuracyScores:
        print(name,':',score)   
    
    # print()
    # print('Classification Reports:-')    
    # for name, score in ClassificationReports:
    #     print(name,':')
    #     print(score)
    #     print()

    return predict

##Evaluation

In [0]:
from sklearn.model_selection import train_test_split

xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size = 0.5)
predicted = ModellingAndEvaluationClassification(xTrain,xTest,yTrain,yTest)

In [0]:
predicted = ModellingAndEvaluationClassification(x,test,y,gndr['Survived'])

In [120]:
from sklearn.model_selection  import KFold,cross_val_score,cross_val_predict

model = LogisticRegression(solver='lbfgs')
kfold = KFold(n_splits=10, random_state=0)

scores =   cross_val_score(model,x,y,scoring='accuracy', cv=kfold)
print(scores)

# predic = cross_val_predict(model,x,y,cv = kfold)
# print(accuracy_score(y,predic))


[0.78888889 0.84269663 0.76404494 0.84269663 0.73033708 0.7752809
 0.7752809  0.75280899 0.86516854 0.80898876]


In [0]:
import lightgbm as lgb

params = { 'objective': 'multiclassova', 'metric': 'multi_logloss', 'verbose': -1, 'num_class':10}
evals_result = {} # to record eval results for plotting

lgb_train = lgb.Dataset(x, y)
# lgb_test = lgb.Dataset(test, None, reference=lgb_train)

In [0]:
gbm = lgb.train(params,lgb_train)

In [108]:
predict = gbm.predict(test)
predict1 = [None]*len(predict)
for i in range(len(predict)) : predict1[i] = np.argmax(predict[i])
accuracy_score(gndr['Survived'],predict1)

0.854066985645933

In [113]:
pr = pd.DataFrame(index = gndr['PassengerId'])
pr['Survived'] = predicted
pr['Survived'].value_counts()

0    263
1    155
Name: Survived, dtype: int64

In [114]:
pr.to_csv('titanic.csv')
!kaggle competitions submit -c titanic -f titanic.csv -m "Message"

100% 2.77k/2.77k [00:03<00:00, 880B/s]
Successfully submitted to Titanic: Machine Learning from Disaster